## Utils


In [1]:
import random

In [3]:
LAMBDA = 1


class BikeParams:
    def __init__(self, quantum_security_level = LAMBDA, bike_variant = 1 | 2):
        if bike_variant in [1, 2]:
            if quantum_security_level == 1:
                self.n, self.r, self.w, self.t = 20326, 10163, 142, 134
            if quantum_security_level == 3:
                self.n, self.r, self.w, self.t = 39706, 19853, 206, 199
            if quantum_security_level == 5:
                self.n, self.r, self.w, self.t = 65498, 32749, 274, 264
        
        else:
            if quantum_security_level == 1:
                self.n, self.r, self.w, self.t = 22054, 11027, 134, 154
            if quantum_security_level == 3:
                self.n, self.r, self.w, self.t = 43366, 21683, 198, 226
            if quantum_security_level == 5:
                self.n, self.r, self.w, self.t = 72262, 36131, 266, 300
            
BIKE = BikeParams()


In [17]:


class Utils:

    def __init__(self) -> None:
        pass

    def odd_div_2(r = BIKE.r)->int:
        if (r // 2) % 2 == 0:
            return r//2 + 1
        return r//2




class BinaryPolynom:  # Polynom or Vector
    def __init__(self, r=BIKE.r) -> None:
        self.degree = r-1
        self.coefficients = [0 for i in range(self.degree+1)]
        pass

    def from_list(self, l):
        self.coefficients = [element for element in l]

    def get_random_polynom_with_weight(self, w = BIKE.w//2):
        self.coefficients[:w//2] = [1 for i in range(w)]
        random.shuffle(self.coefficients)

    def hamming_weight(self):
        return self.coefficients.count(1)

    def transpose(self):
        self.coefficients[1:] = self.coefficients[1:][::-1]
    
    def __str__(self) -> str:
        return "".join(str(bit) for bit in  self.coefficients)
    
    def __repr__(self) -> str:
        return str(self.coefficients)

    def __add__(a, b):
        c = BinaryPolynom(a.degree+1)
        for i in range(a.degree+1):
            c.coefficients[i] = a.coefficients[i] ^ b.coefficients[i]
        return c

    def __eq__(self, __value: object) -> bool:

        if not isinstance(__value, BinaryPolynom):
            return False
        if self.degree != __value.degree:
            return False
        
        for i in range(self.degree+1):
            if self.coefficients[i] != __value.coefficients[i]:
                return False
        return True
    
    def __sub__(a, b):
        c = BinaryPolynom(a.degree+1)
        for i in range(a.degree+1):
            c.coefficients[i] = a.coefficients[i] ^ b.coefficients[i]
        return c

    def __mul__(a, b):
        c = BinaryPolynom(a.degree+1)

        for k in range(b.degree+1):
            for j in range(k+1):
                c.coefficients[k] ^= a.coefficients[j]&b.coefficients[k-j]
            
        return c
        


In [5]:
a = BinaryPolynom(25)
a.get_random_polynom_with_weight(17)

b = BinaryPolynom(25)
b.get_random_polynom_with_weight(16)



print(a)
a.transpose()
print(a)


0000111010000010100000101
0101000001010000010111000


## Key Generation

In [18]:
h_0 = BinaryPolynom()
h_0.get_random_polynom_with_weight()

h_1 = BinaryPolynom()
h_1.get_random_polynom_with_weight()

secret_key = (h_0, h_1)

g = BinaryPolynom()
g.get_random_polynom_with_weight(Utils.odd_div_2()) # BIKE 1

public_key = (h_1*g, h_0*g) # BIKE 1 & BIKE 3

## Encryption

In [7]:
import hashlib

# Generer e0 et e1 deux polynomes tels que |e0| + |e1| = t

def brute_force_generator(t = BIKE.t):

    is_ok = False
    e0, e1 = BinaryPolynom(), BinaryPolynom()
    while not is_ok:
        e0.get_random_polynom_with_weight(random.randint(1, t))
        e1.get_random_polynom_with_weight(random.randint(1, t))
        is_ok = (e0.hamming_weight() + e1.hamming_weight() == t)
    
    return e0, e1

def brute_force_generator_2(t = BIKE.t):
    k = random.randint(1, t)
    e0, e1 = BinaryPolynom(), BinaryPolynom()
    e0.get_random_polynom_with_weight(k)
    e1.get_random_polynom_with_weight(t-k)
    
    return e0, e1

def K(e_0, e_1):
    bit_string = str(e_0) + str(e_1)
    hash = hashlib.sha3_256()
    hash.update(bit_string.encode())

    return hash.digest()[:32]

In [19]:
m = BinaryPolynom()
m.get_random_polynom_with_weight(random.randint(1, BIKE.r))

e_0, e_1 = brute_force_generator_2()
c_0, c_1 = m*public_key[0] + e_0, m*public_key[1] + e_1  # BIKE 1

symmetric_key = K(e_0, e_1)

## Decription

In [15]:
# Définir les paramètres r, w, t, u et delta
r = BIKE.r # La longueur du code
w = BIKE.w # Le poids des lignes de la matrice de parité
t = BIKE.t # Le poids du vecteur d'erreur
u = 0 # Le seuil de poids du syndrome résiduel
delta = 4 # Le nombre de seuils variables

def poly_mul(a, b):
    u = list(map(int, a))
    v = list(map(int, b))
    c = [0 for i in range(len(a))]
    for k in range( len(a)):
        for i in range(k+1):
            c[k] ^= u[i] & v[k-i]
    return "".join(map(str, c))


def shift(a, n):
    c = '0' * n + a
    return c[:n]


# Définir une fonction qui calcule le syndrome résiduel
def residual_syndrome(s, e, h0, h1):
  # Calculer le produit eHT
  eHT = poly_mul(e, h0) + poly_mul(shift(e, 1), h1) # Utiliser le + pour le xor
  # Soustraire eHT à s
  s0 = ''.join([str(int(x) ^ int(y)) for x, y in zip(s, eHT)]) # Utiliser le xor
  # Retourner le syndrome résiduel
  return s0

# Définir une fonction qui calcule le nombre de conflits
def ctr(s, h0, h1, i):
  # Calculer le produit du syndrome et du polynôme décalé
  prod = poly_mul(s, h0 + shift(h1, i)) # Utiliser le + pour le xor
  # Retourner le poids de Hamming du produit
  return hamming_weight(prod)

# Définir une fonction qui calcule le poids de Hamming d'un polynôme
def hamming_weight(a):
  # Compter le nombre de bits à 1 dans le polynôme
  return a.count('1')

# Définir une fonction qui calcule le seuil variable
def threshold(l, s, e):
  # Calculer le poids du syndrome et du vecteur d'erreur
  ws = hamming_weight(s)
  we = hamming_weight(e)
  # Retourner le seuil selon la formule
  return (ws + we) // (2 * (delta - l))

# Définir une fonction qui vérifie les bits du vecteur d'erreur
def check(s, h0, h1, J, T):
  # Initialiser le vecteur d'erreur à zéro
  e = '0' * r
  # Pour chaque indice dans J
  for i in J:
    # Calculer le nombre de conflits
    c_i = ctr(s, h0, h1, i)
    # Si le nombre de conflits est supérieur ou égal à T
    if c_i >= T:
      # Retourner le bit
      e = flip(e, i)
  # Retourner le vecteur d'erreur
  return e

# Définir une fonction qui retourne un bit du polynôme
def flip(a, i):
  # Inverser le bit à la position i
  if a[i] == '0':
    a = a[:i] + '1' + a[i+1:]
  else:
    a = a[:i] + '0' + a[i+1:]
  # Retourner le polynôme modifié
  return a

# Définir une fonction qui devine la position d'un bit erroné
def guess_error_pos(s, h0, h1, T):
  # Répéter jusqu'à trouver un bit erroné
  while True:
    # Tirer un indice aléatoire
    i = random.randint(0, r-1)
    # Calculer le nombre de conflits
    c_i = ctr(s, h0, h1, i)
    # Si le nombre de conflits est supérieur ou égal à T
    if c_i >= T:
      # Retourner l'indice
      return i

# Définir une fonction qui décode un syndrome avec l'algorithme Black Gray Flip
def decode(s, h0, h1):
  # Initialiser le vecteur d'erreur à zéro
  e = '0' * r
  # Calculer le syndrome résiduel
  s0 = residual_syndrome(s, e, h0, h1)
  # Calculer le seuil fixe T
  T = t // 2
  # Pour chaque bit du vecteur d'erreur
  for i in range(r):
    # Calculer le nombre de conflits
    c_i = ctr(s0, h0, h1, i)
    # Si le nombre de conflits est supérieur ou égal à T
    if c_i >= T:
      # Retourner le bit
      e = flip(e, i)
      # Mettre à jour le syndrome résiduel
      s0 = residual_syndrome(s, e, h0, h1)
  # Si le syndrome résiduel est nul ou inférieur à u
  if hamming_weight(s0) <= u:
    # Retourner le vecteur d'erreur
    return e
  # Sinon, effectuer des étapes supplémentaires
  else:
    # Pour chaque seuil variable de 0 à delta
    for l in range(delta):
      # Calculer le seuil variable Tl
      Tl = threshold(l, s0, e)
      # Vérifier les bits du vecteur d'erreur
      e0 = check(s0, h0, h1, range(r), Tl)
      # Mettre à jour le vecteur d'erreur et le syndrome résiduel
      e = ''.join([str(int(x) ^ int(y)) for x, y in zip(e, e0)]) # Utiliser le xor
      s0 = residual_syndrome(s, e, h0, h1)
      # Si le syndrome résiduel est nul ou inférieur à u
      if hamming_weight(s0) <= u:
        # Retourner le vecteur d'erreur
        return e
    # Tant que le syndrome résiduel est supérieur à u
    while hamming_weight(s0) > u:
      # Deviner la position d'un bit erroné
      i = guess_error_pos(s0, h0, h1, T)
      # Retourner le bit
      e = flip(e, i)
      # Mettre à jour le syndrome résiduel
      s0 = residual_syndrome(s, e, h0, h1)
    # Retourner le vecteur d'erreur
    return e


In [16]:
# calcule du syndrome
s = c_0*h_0 + c_1*h_1
u = 0

# Decoder s pour retrouver (e_0_prime, e_1_prime)

e = decode(str(s), str(h_0), str(h_1))


KeyboardInterrupt: 

In [ ]:
tau = 3
NBIter = 5

# Input: H is an r x n matrix over F2, s is an r-bit vector, h0 and h1 are w/2-bit vectors
# Output: e = (e0, e1) is an n-bit vector with Hamming weight t, or None if decoding fails
def BGF_decoder(H, s, h0, h1):
  # Initialize the error vector e and the lists black and gray
  e = [0] * BIKE.n # n-bit vector of zeros
  black = [] # empty list
  gray = [] # empty list
  # Iterate NBIter times
  for i in range(NBIter):
    # Compute the threshold T based on the Hamming weight of s + e*H^T
    T = threshold(sum(s + e * H.T))
    # Call the BFIter function to flip bits in e and update black and gray
    e, black, gray = BFIter(H, s, e, T, black, gray)
    # If black is empty, return e
    if not black:
      return e
    # Call the BFMIter function to flip bits in e using black and gray
    e = BFMIter(H, s, e, black, gray)
    # If e has Hamming weight t, return e
    if sum(e) == t:
      return e
  # If all iterations fail, return None
  return None

# Input: H, s, e, T, black, gray as in BGF_decoder
# Output: e, black, gray updated after one iteration
def BFIter(H, s, e, T, black, gray):
  # Initialize the new lists black and gray
  black_new = []
  gray_new = []
  # For each column of H
  for j in range(n):
    # Compute the counter value C_j as the Hamming weight of H_j * s
    C_j = sum(H[j] * s)
    # If C_j is greater than or equal to T
    if C_j >= T:
      # Flip the j-th bit of e
      e[j] = 1 - e[j]
      # Append j to black_new
      black_new.append(j)
    # Else if C_j is greater than or equal to T - tau
    elif C_j >= T - tau:
      # Append j to gray_new
      gray_new.append(j)
  # Return e, black_new, gray_new
  return e, black_new, gray_new

# Input: H, s, e, black, gray as in BGF_decoder
# Output: e updated after one iteration
def BFMIter(H, s, e, black, gray):
  # For each element j in black
  for j in black:
    # Flip the j-th bit of e
    e[j] = 1 - e[j]
    # Compute the counter value C_j as the Hamming weight of H_j * s
    C_j = sum(H[j] * s)
    # If C_j is less than T - tau
    if C_j < T - tau:
      # Flip the j-th bit of e back
      e[j] = 1 - e[j]
  # For each element j in gray
  for j in gray:
    # Flip the j-th bit of e
    e[j] = 1 - e[j]
    # Compute the counter value C_j as the Hamming weight of H_j * s
    C_j = sum(H[j] * s)
    # If C_j is greater than or equal to T
    if C_j >= T:
      # Flip the j-th bit of e back
      e[j] = 1 - e[j]
  # Return e
  return e


In [ ]:


symmetric_key_decapsuled = bytes()
e_0_prime, e_1_prime = BinaryPolynom(), BinaryPolynom()
if e_0_prime.hamming_weight()  + e_1_prime.hamming_weight() != BIKE.t:
    print("Decapsulation error !")
    
else:
    symmetric_key_decapsuled = K(e_0_prime, e_1_prime)
 

## BIKE Entry point

In [ ]:

# Fonction principale pour exécuter les fonctionnalités de BIKE
def main():
    # Générer les clés publiques et privées
    public_key, private_key = key_generation.generate_key_pair()

    # Message à chiffrer
    message = "Hello, World!"

    # Chiffrement du message avec la clé publique
    ciphertext = encryption.encrypt(message, public_key)

    # Déchiffrement du message chiffré avec la clé privée
    decrypted_message = decryption.decrypt(ciphertext, private_key)

    # Affichage des résultats
    print("Message clair :", message)
    print("Message chiffré :", ciphertext)
    print("Message déchiffré :", decrypted_message)

# Exécution de la fonction principale
if __name__ == "__main__":
    main()